In [2]:
import os
import re
import numpy as np
import shutil

In [3]:
os.getcwd()

'/Users/vogg/Documents/Python/interaction-labelling-tool'

In [4]:
path = "/Users/vogg/Documents/Labeling/Macaques/macaque_videos_eval/"

In [5]:
video_list = os.listdir(path)
video_list = [elem for elem in video_list if elem.startswith("VID_")]

In [6]:
video = video_list[0]
video

'VID_20210227_133251_0'

In [7]:
with open(path + video + "/obj.names", "w") as f:
    f.write("macaque")


In [8]:
with open(path + video + "/obj.data", "w") as f:
    f.write("classes = 1\n")
    f.write("train = data/train.txt\n")
    f.write("names = data/obj.names\n")
    f.write("backup = backup/")



In [9]:
with open(path + video + "/train.txt", "w") as f:
    for file in sorted(os.listdir(path + video + "/obj_train_data")):
        if file.endswith("txt"):
            f.write("data/obj_train_data/" + file.replace(".txt", ".jpg") + "\n")


In [10]:
def read_agecker_Macaque_wild(folder_path, scale_size = (1920, 1080)):
    width, height = scale_size
    data_names = os.listdir(folder_path)
    data_names = [name for name in data_names if re.search(".txt",name)]
    frame_nr = []
    class_name = []
    id_nr = []
    bbox_xywh = []
    for data_name in data_names:
        maca_data = open(os.path.join(folder_path,data_name)).read().strip().split("\n")
        if maca_data == [""]:
            continue
        maca_data = [[float(nr) for nr in row.strip().split(" ")] for row in maca_data]
        frame_name = data_name.split(".")[0]
        #-1 that mot also starts at 0
        frame_nr_to_append = int(re.findall("[0-9]+",frame_name)[0])
        for row in maca_data:
            frame_nr.append(frame_nr_to_append)
            class_name.append(row[0])
            id_nr.append(row[1])
            #to upper left corner from middle 
            row[2] = (float(row[2]) - float(row[4])/2) * width
            row[3] = (float(row[3]) - float(row[5])/2) * height
            row[4] = float(row[4]) * width
            row[5] = float(row[5]) * height
            row[2:6] = [round(x,2) for x in row[2:6]]
            bbox_xywh.append(row[2:6])
    #now bring them in the right order of the frames for sure:
    order_to_follow = np.argsort(frame_nr)
    frame_nr = [frame_nr[i] for i in order_to_follow]
    id_nr = [id_nr[i] for i in order_to_follow]
    bbox_xywh = [bbox_xywh[i] for i in order_to_follow]
    class_name = [class_name[i] for i in order_to_follow]
    return {"frame_name":frame_nr,"id_nr":id_nr,"bbox_xywh":bbox_xywh,"class_name":class_name}



def write_MOT(path, dict_to_write):
    """
    Write the MOT file for the given dictionary

    path: path to write the file
    dict_to_write: dictionary with MOT data
    """
    frame_names = dict_to_write["frame_name"]
    frame_nr = [int(re.findall("[0-9]+", str(frame_name))[0]) for frame_name in frame_names]
    id_nr = dict_to_write["id_nr"]
    bbox_xywh = dict_to_write["bbox_xywh"]
    #confidence_score = dict_to_write["confidence_score"]
    n = len(frame_nr)

    with open(path, "w") as f:
        for i in range(n -1 ):
            row = [str(frame_nr[i]+1), str(int(id_nr[i])+1), *[str(round(x, 2)) for x in bbox_xywh[i]], str(1),str(1),str(1.0)]
            f.write(",".join(row) + "\n")

        row = [str(frame_nr[n-1]+1), str(int(id_nr[n-1])+1), *[str(round(x, 2)) for x in bbox_xywh[n-1]], str(1),str(1),str(1.0)]
        f.write(",".join(row))

In [11]:
def zip_folder(folder_path):
    # Get the parent directory and name of the folder
    parent_dir = os.path.dirname(folder_path)
    folder_name = os.path.basename(folder_path)
    
    # Create the zip file name
    zip_filename = f"{folder_name}.zip"
    zip_filepath = os.path.join(parent_dir, zip_filename)
    
    # Zip the folder
    shutil.make_archive(folder_path, 'zip', folder_path)
    
    # Delete the folder
    shutil.rmtree(folder_path)
    
    print(f"Folder '{folder_path}' zipped and deleted. Zip file saved as '{zip_filepath}'.")


In [12]:
for video in video_list:
    if not os.path.exists(os.path.join(path, video, "gt")):
        os.mkdir(os.path.join(path, video, "gt"))
    full_path = os.path.join(path, video, "labels_with_ids")
    test_dict = read_agecker_Macaque_wild(full_path)
    write_MOT(os.path.join(path, video, "gt/gt.txt"), test_dict)

    with open(os.path.join(path, video, "gt/labels.txt"), "w") as f:
        f.write("macaque")
    #zip_folder(os.path.join(path, video, "gt/"))

In [63]:
for video in video_list:
    shutil.rmtree(os.path.join(path, video, "test"))